In [ ]:
from pyannote.audio import Pipeline
import pickle
import os
import time
import soundfile as sf
from pyannote.core.segment import Segment

In [ ]:
def cropp_wav_from_segments(audiofile_path, output_folder, probant_segments, probant_acronym):

    for i, segment in enumerate(probant_segments):
        
        wav_data, wav_samplerate = sf.read(audiofile_path)
        start_sample = int(segment.start * wav_samplerate)  # Assuming you have the sample rate stored in 'samplerate'
        end_sample = int(segment.end * wav_samplerate)
        cropped_data = wav_data[start_sample:end_sample]

        # Save the cropped segment to a new WAV file
        output_file_path = os.path.join(output_folder, f'{probant_acronym}_Interview_{i}.wav')
        sf.write(output_file_path, cropped_data, wav_samplerate)

    print(f"Safed cropped segments for {probant_acronym}")

In [ ]:
def load_annotation_objects(probant_acronym):
    
    filename_diarization = f"{probant_acronym}_diarization.pkl"
    filepath = os.path.join('../data/annotation_objects/', filename_diarization)
    with open(filepath, 'rb') as file:
        diarization = pickle.load(file)
        
    filename_interviewer_segments = f"{probant_acronym}_interviewer_segments.pkl"
    filepath = os.path.join('../data/annotation_objects/', filename_interviewer_segments)
    with open(filepath, 'rb') as file:
        interviewer_segments = pickle.load(file)
    
    filename_probant_segments = f"{probant_acronym}_probant_segments.pkl"
    filepath = os.path.join('../data/annotation_objects/', filename_probant_segments)
    with open(filepath, 'rb') as file:
        probant_segments = pickle.load(file)
        
    return (diarization, interviewer_segments, probant_segments)

In [ ]:
# consider all available diarizations
diarized_probant_interviews = []
seen_substrings = set()
for file in os.listdir('../data/annotation_objects/'):
    # Finde das zweite Vorkommen des Unterstrichs
    index = file.find('_', file.find('_') + 1)
    if index != -1:
        substring = file[:index]
        if substring not in seen_substrings:
            diarized_probant_interviews.append(substring)
            seen_substrings.add(substring)
            
output_folder = '../data/cropped_interview_data/all_segments(not_checked)'

for probant_acronym in diarized_probant_interviews:
    
    diarization, interviewer_segments, probant_segments = load_annotation_objects(probant_acronym)
    
    probant = probant_acronym.split('_')[0]
    timepoint = probant_acronym.split('_')[1]
    audiofile_folder = f'../data/20221121_Sprachaufnahmen_Preprocessed/{probant}_Interview_{timepoint}'
    for file_name in os.listdir(audiofile_folder):
        if 'ZOOM' in file_name and not file_name.endswith('BU.WAV') and (file_name.endswith('.WAV') or file_name.endswith('.wav')):
            filename = file_name
            continue
    audiofile_path = os.path.join(audiofile_folder, filename)
    print(filename)
    cropp_wav_from_segments(audiofile_path, output_folder, probant_segments, probant_acronym)
    